In [33]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import pandas as pd
from pandas.io.json import json_normalize

In [0]:
file_path = "gdrive/My Drive/training_mixpanel.txt"

In [0]:
import json

with open(file_path, encoding='utf-8-sig') as json_file:
    json_data = json.load(json_file)

In [0]:
def normalize_data(df):
    df_matrix = pd.pivot_table(df, values='quantity', index='customer_id', columns='product_id')
    df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
    d = df_matrix_norm.reset_index()
    d.index.names = ['scaled_purchase_freq']
    return pd.melt(d, id_vars=['customer_id'], value_name='scaled_purchase_freq').dropna()

In [0]:
customers=json_normalize(json_data)
customers.columns = ['event', 'country', 'customer_id', 'description', 'invoice_date', 'invoice_no', 'product_id', 'quantity', 'unit_price']

In [0]:
customers

In [0]:
df = json_normalize(json_data)



df.columns = ['event', 'country', 'customer_id', 'description', 'invoice_date', 'invoice_no', 'product_id', 'quantity', 'unit_price']

In [0]:
df = df.drop(['event', 'invoice_date', 'invoice_no'], axis=1)


In [0]:
df = normalize_data(df)

In [0]:
from sklearn.model_selection import train_test_split

In [45]:
!pip install turicreate

NotImplementedError: ignored

In [0]:
import time
import turicreate as tc


In [0]:
def split_data(data):
  train, test = train_test_split(data, test_size = .2)
  
  train_data = tc.SFrame(train)
  
  test_data = tc.SFrame(test)
  
  return train_data, test_data

In [0]:
train_data_norm, test_data_norm = split_data(df)

In [49]:
train_data_norm

customer_id,product_id,scaled_purchase_freq
17673,22434,0.014084507042253521
14505,22495,0.0
17965,84380,0.027777777777777776
13050,22464,0.06735751295336788
15241,21791,0.058823529411764705
14704,21466,0.0
17611,22819,0.08045977011494253
13263,23207,0.011666666666666667
17757,21207,0.10638297872340426
17472,79030G,0.021739130434782608


In [0]:
user_id = 'customer_id'
item_id = 'product_id'
users_to_recommend = list(customers[user_id].unique())
n_rec = 3389 # number of items to recommend
n_display = 30 # to display the first few rows in an output dataset

In [0]:
def model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display):
  model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='cosine')
  recom = model.recommend(users=users_to_recommend, k=n_rec)
  recom.print_rows(n_display)
  return model

In [52]:
name = 'cosine' 
target = 'scaled_purchase_freq' 
cos_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 170528 observations with 4338 users and 3379 items.

Data prepared in: 0.212272s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 1000/4363 queries. users per second: 1046.28

recommendations finished on 2000/4363 queries. users per second: 1033.84

recommendations finished on 3000/4363 queries. users per second: 1043.19

recommendations finished on 4000/4363 queries. users per second: 1046.81

+-------------+------------+----------------------+------+
| customer_id | product_id |        score         | rank |
+-------------+------------+----------------------+------+
|    17850    |   82494L   | 0.013867782694952828 |  1   |
|    17850    |   82482    | 0.013744158404214042 |  2   |
|    17850    |   71496A   | 0.013167002371379308 |  3   |
|    17850    |   20655    | 0.012838461569377355 |  4   |
|    17850    |   22903    | 0.012233785220554896 |  5   |
|    17850    |   35923    | 0.011563088212694441 |  6   |
|    17850    |   84884A   | 0.011485436132975988 |  7   |
|    17850    |   20679    | 0.011073644672121321 |  8   |
|    17850    |   35095A   | 0.010863453149795532 |  9   |
|    17850    |   90057    | 0.010838632072721208 |  10  |
|    17850    |   90210D   | 0.01077385459627424  |  11  |
|    17850    |   35095B   |  0.010728691305433   |  12  |
|    17850    |   21733    | 0.010722024100167411 |  13  |
|    17850    |   47481    | 0.010720095464161463 |  14 

In [0]:
test_data_norm.shape

In [64]:
cos_norm.evaluate( test_data_norm)

recommendations finished on 1000/3978 queries. users per second: 11638.3

recommendations finished on 2000/3978 queries. users per second: 11689.6

recommendations finished on 3000/3978 queries. users per second: 11636.5


Precision and recall summary statistics by cutoff
+--------+----------------------+----------------------+
| cutoff |    mean_precision    |     mean_recall      |
+--------+----------------------+----------------------+
|   1    | 0.03921568627450978  | 0.008679250347307717 |
|   2    | 0.029663147310206094 | 0.013018532652538228 |
|   3    | 0.024635495223730538 | 0.016006329821276772 |
|   4    | 0.02155605832076417  | 0.01835183677222756  |
|   5    | 0.01945701357466061  | 0.02107676479486912  |
|   6    | 0.017471091000502818 | 0.023003226974747087 |
|   7    | 0.01612439847733962  | 0.02445312107589168  |
|   8    | 0.01505153343388638  | 0.02570485814865485  |
|   9    | 0.013881906038768772 | 0.026131461829334547 |
|   10   | 0.013247863247863246 | 0.027490957436983487 |
+--------+----------------------+----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.24816302102822008

Per User RMSE (best)
+-------------+------+-------+
| customer_id | rmse | count |
+--------

In [62]:
final_model = tc.item_similarity_recommender.create(tc.SFrame(df), 
                                            user_id=user_id, 
                                            item_id=item_id, 
                                            target=target, similarity_type='cosine')

recom = final_model.recommend(users=users_to_recommend, k=n_rec)
recom.print_rows(n_display)

Preparing data set.

Data has 213160 observations with 4363 users and 3389 items.

Data prepared in: 0.262373s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 1000/4363 queries. users per second: 759.544

recommendations finished on 2000/4363 queries. users per second: 861.812

recommendations finished on 3000/4363 queries. users per second: 905.191

recommendations finished on 4000/4363 queries. users per second: 927.719

+-------------+------------+----------------------+------+
| customer_id | product_id |        score         | rank |
+-------------+------------+----------------------+------+
|    17850    |   71496A   | 0.00924374075496898  |  1   |
|    17850    |   72232    | 0.00918054931304034  |  2   |
|    17850    |   16216    | 0.00912618637084961  |  3   |
|    17850    |   47481    | 0.009125225684222053 |  4   |
|    17850    |   16151A   | 0.009093137348399442 |  5   |
|    17850    |   44234    | 0.009052669300752528 |  6   |
|    17850    |   20655    | 0.009011748959036434 |  7   |
|    17850    |   35095A   | 0.008990452570073745 |  8   |
|    17850    |   35095B   | 0.008897518410402186 |  9   |
|    17850    |   22903    | 0.008757405421313117 |  10  |
|    17850    |   90210D   | 0.008663507068858427 |  11  |
|    17850    |   21634    | 0.008663447464213651 |  12  |
|    17850    |   21733    | 0.008637635146870333 |  13  |
|    17850    |   90057    | 0.008602100260117474 |  14 

In [63]:
df_rec = recom.to_dataframe()
print(df_rec.shape)
df_rec.head(100)

(14573047, 4)


,customer_id,product_id,score,rank
0,17850,71496A,0.009244,1
1,17850,72232,0.009181,2
2,17850,16216,0.009126,3
3,17850,47481,0.009125,4
4,17850,16151A,0.009093,5
5,17850,44234,0.009053,6
6,17850,20655,0.009012,7
7,17850,35095A,0.008990,8
8,17850,35095B,0.008898,9
9,17850,22903,0.008757,10


In [0]:
df_rec['customer_id'].unique()

In [0]:
df_rec=df_rec.drop_duplicates()

In [0]:
df_rec.count()

In [0]:
#df_rec.to_csv('rec.csv', encoding = 'utf-8')


In [0]:
c=df_rec.copy()

In [0]:
df1=c.pivot(index='customer_id',columns='product_id',values='rank')

In [0]:
df1.to_csv('ex.csv',encoding='utf-8')

In [0]:
models_w_norm = [cos_norm]

In [0]:
names_w_norm = ['Cosine Similarity on Scaled Purchase Counts']

In [60]:
eval_norm = tc.recommender.util.compare_models(test_data_norm, models_w_norm, model_names=names_w_norm)

PROGRESS: Evaluate model Cosine Similarity on Scaled Purchase Counts


recommendations finished on 1000/3978 queries. users per second: 10105.2

recommendations finished on 2000/3978 queries. users per second: 9992.95

recommendations finished on 3000/3978 queries. users per second: 9955.14


Precision and recall summary statistics by cutoff
+--------+----------------------+----------------------+
| cutoff |    mean_precision    |     mean_recall      |
+--------+----------------------+----------------------+
|   1    | 0.039215686274509776 | 0.008679250347307732 |
|   2    | 0.029663147310206174 | 0.01301853265253825  |
|   3    | 0.024635495223730527 | 0.01600632982127672  |
|   4    | 0.02155605832076422  | 0.018351836772227605 |
|   5    | 0.01940673705379585  | 0.021055816244508804 |
|   6    | 0.017471091000502762 | 0.023019187775021634 |
|   7    | 0.01612439847733962  | 0.024469081876166147 |
|   8    | 0.015051533433886393 | 0.025704858148654863 |
|   9    | 0.013881906038768769 | 0.02613146182933457  |
|   10   | 0.013247863247863284 | 0.027490957436983494 |
+--------+----------------------+----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.24816302102822008

Per User RMSE (best)
+-------------+------+-------+
| customer_id | rmse | count |
+--------